In [2]:
%run read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d31d3498-2c26-4bb2-ace2-b75351639438;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 287ms :: artifacts dl 15ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------

23/03/13 01:32:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/13 01:32:54 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [7]:
%run build_adjacency.ipynb

In [8]:
df_epm = readCheckpoint(size = "medium")

 1:48AM UTC on Mar 13, 2023 --- read and cache time: 1.0855274200439453 seconds ---


In [12]:
df_eps.printSchema()

root
 |-- id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- objectID: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- event_minute: integer (nullable = true)
 |-- event_day: integer (nullable = true)
 |-- event_hour: integer (nullable = true)



In [16]:
print(df_eps.count())
df_eps.select("object").distinct().show()
df_eps.select("event_day").distinct().show()
df_eps.select("event_hour").distinct().show()

10000000
+-------+
| object|
+-------+
|  SHELL|
|   FLOW|
|   FILE|
|PROCESS|
+-------+

+---------+
|event_day|
+---------+
|       23|
+---------+

+----------+
|event_hour|
+----------+
|        16|
|        15|
|        17|
|        18|
|        13|
|        14|
|        19|
+----------+



In [3]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml import Pipeline

# Create a PySxpark DataFrame
df = spark.createDataFrame([(0, "category1"), (1, "category2"), (2, "category3"), (3, "category1")], ["id", "category"])


In [4]:
df.toPandas()

,id,category
0,0,category1
1,1,category2
2,2,category3
3,3,category1


In [9]:
from pyspark.ml.feature import StringIndexer

#turn into numeric index before encoding
indexer = StringIndexer(inputCol='category', outputCol='class_numeric')
indexer_fitted = indexer.fit(df)
df_indexed = indexer_fitted.transform(df)
df_indexed.show()

encoder = OneHotEncoder(inputCols=['class_numeric'], outputCols=['class_onehot'],dropLast=False)
df_onehot = encoder.fit(df_indexed).transform(df_indexed)
df_onehot.show()

+---+---------+-------------+
| id| category|class_numeric|
+---+---------+-------------+
|  0|category1|          0.0|
|  1|category2|          1.0|
|  2|category3|          2.0|
|  3|category1|          0.0|
+---+---------+-------------+

+---+---------+-------------+-------------+
| id| category|class_numeric| class_onehot|
+---+---------+-------------+-------------+
|  0|category1|          0.0|(3,[0],[1.0])|
|  1|category2|          1.0|(3,[1],[1.0])|
|  2|category3|          2.0|(3,[2],[1.0])|
|  3|category1|          0.0|(3,[0],[1.0])|
+---+---------+-------------+-------------+



In [12]:
#now turn sparse array into columns
from pyspark.ml.functions import vector_to_array
df_col_onehot = df_onehot.select('*', vector_to_array('class_onehot').alias('col_onehot'))
df_col_onehot.show()

#and then unpack vectors into columns
num_categories = len(df_col_onehot.first()['col_onehot'])   # 3
cols_expanded = [(col('col_onehot')[i]) for i in range(num_categories)]
df_cols_onehot = df_col_onehot.select('id', 'category', *cols_expanded)
df_cols_onehot.show()

+---+---------+-------------+-------------+---------------+
| id| category|class_numeric| class_onehot|     col_onehot|
+---+---------+-------------+-------------+---------------+
|  0|category1|          0.0|(3,[0],[1.0])|[1.0, 0.0, 0.0]|
|  1|category2|          1.0|(3,[1],[1.0])|[0.0, 1.0, 0.0]|
|  2|category3|          2.0|(3,[2],[1.0])|[0.0, 0.0, 1.0]|
|  3|category1|          0.0|(3,[0],[1.0])|[1.0, 0.0, 0.0]|
+---+---------+-------------+-------------+---------------+

+---+---------+-------------+-------------+-------------+
| id| category|col_onehot[0]|col_onehot[1]|col_onehot[2]|
+---+---------+-------------+-------------+-------------+
|  0|category1|          1.0|          0.0|          0.0|
|  1|category2|          0.0|          1.0|          0.0|
|  2|category3|          0.0|          0.0|          1.0|
|  3|category1|          1.0|          0.0|          0.0|
+---+---------+-------------+-------------+-------------+



In [13]:
#now rename the categories
cols_expanded = [col('col_onehot')[i].alias(f'{indexer_fitted.labels[i]}') for i in range(num_categories)]
df_cols_onehot = df_col_onehot.select('id', 'category', *cols_expanded)
df_cols_onehot.show()

+---+---------+---------+---------+---------+
| id| category|category1|category2|category3|
+---+---------+---------+---------+---------+
|  0|category1|      1.0|      0.0|      0.0|
|  1|category2|      0.0|      1.0|      0.0|
|  2|category3|      0.0|      0.0|      1.0|
|  3|category1|      1.0|      0.0|      0.0|
+---+---------+---------+---------+---------+



In [15]:
from pyspark.ml.feature import BucketedRandomProjectionLSH, VectorAssembler

#now create an index based on the sparse vector
# create a BucketedRandomProjectionLSH instance to hash the sparse vector column
brp = BucketedRandomProjectionLSH(inputCol="class_onehot", outputCol="HashedFeatures", numHashTables=3, bucketLength=0.1)
# fit the transformer on the DataFrame and transform it
brp_model = brp.fit(df_onehot)
df_hashed = brp_model.transform(df_onehot)
# show the resulting DataFrame
df_hashed.show()


+---+---------+-------------+-------------+--------------------+
| id| category|class_numeric| class_onehot|      HashedFeatures|
+---+---------+-------------+-------------+--------------------+
|  0|category1|          0.0|(3,[0],[1.0])|[[1.0], [-4.0], [...|
|  1|category2|          1.0|(3,[1],[1.0])|[[8.0], [-7.0], [...|
|  2|category3|          2.0|(3,[2],[1.0])|[[4.0], [6.0], [-...|
|  3|category1|          0.0|(3,[0],[1.0])|[[1.0], [-4.0], [...|
+---+---------+-------------+-------------+--------------------+



In [17]:
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.linalg import Vectors

# create a VectorIndexer instance to assign integers to the unique hashed values
vi = VectorIndexer(inputCol="HashedFeatures", outputCol="IndexedFeatures", maxCategories=3)

# transform the DataFrame using the VectorIndexer
df_indexed = vi.fit(df_hashed).transform(df_hashed)

# show the resulting DataFrame
df_indexed.show()

IllegalArgumentException: requirement failed: Column HashedFeatures must be of type class org.apache.spark.ml.linalg.VectorUDT:struct<type:tinyint,size:int,indices:array<int>,values:array<double>> but was actually class org.apache.spark.sql.types.ArrayType:array<struct<type:tinyint,size:int,indices:array<int>,values:array<double>>>.

In [20]:
df_hashed = df_hashed.withColumn('color',lit(['r','g','r','r']))

Py4JJavaError: An error occurred while calling z:org.apache.spark.sql.functions.lit.
: org.apache.spark.SparkRuntimeException: The feature is not supported: literal for '[r, g, r, r]' of class java.util.ArrayList.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.literalTypeUnsupportedError(QueryExecutionErrors.scala:283)
	at org.apache.spark.sql.catalyst.expressions.Literal$.apply(literals.scala:101)
	at org.apache.spark.sql.functions$.lit(functions.scala:125)
	at org.apache.spark.sql.functions.lit(functions.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4

In [40]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.linalg import Vectors

# create sample data
data = [(0, "red",'g'), (1, "green",'blue'), (2, "red",'g')]

# create dataframe
df = spark.createDataFrame(data, ["id", "color",'second_color'])

df.printSchema()

#now create a hash
df = df.withColumn("sha2",sha2(concat_ws('||',col('color'),col('second_color')),256))

df.printSchema()




root
 |-- id: long (nullable = true)
 |-- color: string (nullable = true)
 |-- second_color: string (nullable = true)

root
 |-- id: long (nullable = true)
 |-- color: string (nullable = true)
 |-- second_color: string (nullable = true)
 |-- sha2: string (nullable = true)



In [41]:
#turn into numeric index before encoding
indexer = StringIndexer(inputCol='sha2', outputCol='class_numeric')
indexer_fitted = indexer.fit(df)
df_indexed = indexer_fitted.transform(df)
df_indexed.show()

+---+-----+------------+--------------------+-------------+
| id|color|second_color|                sha2|class_numeric|
+---+-----+------------+--------------------+-------------+
|  0|  red|           g|1edcfa2380771ecfb...|          0.0|
|  1|green|        blue|50e3429807908a500...|          1.0|
|  2|  red|           g|1edcfa2380771ecfb...|          0.0|
+---+-----+------------+--------------------+-------------+



In [2]:
#load in size medium
df_eps = readCheckpoint(size='medium')

 3:13PM UTC on Mar 10, 2023 --- read and cache time: 9.284032821655273 seconds ---


In [3]:
list_mal_hosts = ['SysClient0559.systemia.com',
'SysClient0255.systemia.com',
'SysClient0205.systemia.com',
'SysClient0503.systemia.com',
'SysClient0355.systemia.com',
'SysClient0104.systemia.com',
'SysClient0874.systemia.com',
'SysClient0955.systemia.com',
'SysClient0201.systemia.com',
'SysClient0402.systemia.com',
'SysClient0660.systemia.com',
'SysClient0321.systemia.com',
'SysClient0609.systemia.com',
'SysClient0771.systemia.com',
'SysClient0462.systemia.com',
'SysClient0419.systemia.com',
'SysClient0010.systemia.com',
'SysClient0069.systemia.com',
'SysClient0358.systemia.com',
'SysClient0203.systemia.com',
'SysClient0618.systemia.com',
'SysClient0851.systemia.com',
'SysClient0501.systemia.com',
'SysClient0811.systemia.com',
'SysClient0170.systemia.com',
'SysClient0351.systemia.com',
'SysClient0051.systemia.com']

In [5]:
df_epm = df_eps.filter(col("hostname").isin(list_mal_hosts))

In [6]:
df_epm.printSchema()

root
 |-- id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- objectID: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- event_minute: integer (nullable = true)
 |-- event_day: integer (nullable = true)
 |-- event_hour: integer (nullable = true)



In [7]:
df_epm.select("event_day").distinct().show()

+---------+
|event_day|
+---------+
|       23|
+---------+



In [10]:
df_epm.count()

6553319

In [9]:
#let's learn about user log-on events
df_epm.where(col('object') == 'USER_SESSION').count()

0

In [11]:
#get a count of malicious events to compare what we have to what we need
df_labels = readCheckpoint_bcm('labels')

 4:20PM UTC on Mar 10, 2023 --- read and cache time: 1.71661376953125e-05 seconds ---


In [12]:
# Create a new dataframe with distinct objectIDs to identify malcious ObjectIds
df_malcious_objectIDs = df_labels.select('id').distinct()

# Create a new column called 'malicious' in df_ecar to label malicious records
df_epm = df_epm.withColumn('malicious', when(col('id').isin(df_malcious_objectIDs.rdd.flatMap(lambda x: x).collect()), 1).otherwise(0))

In [13]:
df_epm.printSchema()

root
 |-- id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- objectID: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- event_minute: integer (nullable = true)
 |-- event_day: integer (nullable = true)
 |-- event_hour: integer (nullable = true)
 |-- malicious: integer (nullable = false)



In [15]:
#how many malicious and how many not
print("malicious: "+ str(df_epm.where(col('malicious') == 1).count()))
print("benign: "+ str(df_epm.where(col('malicious') == 0).count()))

malicious: 71193


benign: 6482126


In [19]:
#so what I am going to need to do is run build adjaceny matrix against this function. 

#first let's pull a small section of the benign
df_epm_benign = df_epm.where(col('malicious') == 0).limit(300000)
df_epm_mal = df_epm.where(col('malicious') == 1)

In [20]:
#work on extracting parent time and subtracting from child time
#get actIDs
col_objID = df_epm_mal.select("objectID").distinct().collect()
col_actID = df_epm_mal.select("actorID").distinct().collect()

#put these columns into a dictionary and turn into a spark data frame
data = [{"actorIDs" : [row.actorID for row in col_actID], \
               "objectIDs" : [row.objectID for row in col_objID]}]

id_intersect = spark.createDataFrame(data)

#compare ID pools to find actIDs that are nodes
id_intersect = id_intersect.withColumn("id_int", \
                                       array_intersect(col("actorIDs"),col("objectIDs")))

#now build a list of the nodal actIDs and print the length
actIDs = [row.id_int for row in id_intersect.select("id_int").collect()][0]
print(len(actIDs))

20


In [34]:
#now pull time and child_event for first actor
first_act = actIDs[1]
birth_event = df_epm_mal.where((col("objectID") == first_act) \
                        & (col("actorID")!= first_act)).limit(3).collect()


In [27]:
print(birth_event[0])

Row(id='e8cd79b8-6d34-4e34-8771-535c38cc5178', timestamp=datetime.datetime(2019, 9, 23, 18, 44, 59, 248000), objectID='2d02ab42-4400-4049-baad-6dfae19c3c4c', actorID='796a63d7-45d9-4a50-8d1c-94a9c4e74e54', object='PROCESS', action='CREATE', hostname='SysClient0609.systemia.com', user_name=None, privileges=None, image_path='powershell.exe', parent_image_path='\\Device\\HarddiskVolume1\\Windows\\system32\\wbem\\wmiprvse.exe', new_path=None, file_path=None, direction=None, logon_id=None, requesting_domain=None, requesting_user=None, event_minute=44, event_day=23, event_hour=18, malicious=1)


In [36]:
#str(row["objectID"]+':'+row["first(id)"])

str_key = str(birth_event[0]['objectID']+":" \
              +str(birth_event[0]['timestamp'])+":" \
              +birth_event[0]['object']+":" \
              +birth_event[0]['parent_image_path']+":" \
              +birth_event[0]['image_path'])
print(str_key)
              

70ba4f9c-e27b-4a34-8dce-303c00c227c2:2019-09-23 18:44:53.785000:PROCESS:\Device\HarddiskVolume1\Windows\system32\wbem\wmiprvse.exe:powershell.exe


In [35]:
for row in birth_event:
    print(row['parent_image_path'])

\Device\HarddiskVolume1\Windows\system32\wbem\wmiprvse.exe


In [13]:
%run build_adjacency.ipynb

In [4]:
df_epm.select("event_hour").where(col("event_day") == 23).distinct().show()

+----------+
|event_hour|
+----------+
|        16|
|        19|
|        15|
|        17|
|        18|
|        13|
|        14|
|        22|
|        21|
|        20|
|        12|
|         9|
|        23|
|         7|
|        10|
|        11|
|         3|
|         5|
|         4|
|         8|
+----------+
only showing top 20 rows



In [5]:
df_epm.select('event_day').distinct().show()

+---------+
|event_day|
+---------+
|       23|
+---------+



In [10]:
df_epm_9_10 = df_epm.where((col('event_hour')== 9) | (col('event_hour')== 10))

In [11]:
df_epm_test = build_adjaceny(df_epm_9_10)

1608


length actIDs: 1308
elapsed time: 2.8200037479400635
ed4dfa35-ee46-4e37-a2b6-9838128b9b50


no birth event for 83 events.
elapsed time 2: 2158.625776529312


In [12]:
s3_url_trusted = "s3a://sapient-bucket-trusted/"
s3_write_loc = f"{s3_url_trusted}/prod/adjacency-graph/0910"
df_epm_test.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(s3_write_loc)

In [10]:
df_epm_11 = df_epm.where((col('event_hour')== 11))
df_epm_11w = build_adjaceny(df_epm_11)

640


length actIDs: 491
elapsed time: 178.3520474433899
5ffe548e-f658-49b9-b448-9d5c78f7788d


no birth event for 41 events.
elapsed time 2: 949.9309887886047


In [11]:
#df_epm_11 = build_adjaceny(df_epm_11)
s3_url_trusted = "s3a://sapient-bucket-trusted/"
s3_write_loc = f"{s3_url_trusted}/prod/adjacency-graph/11"
df_epm_11w.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(s3_write_loc)